# CHEME 5760 Final Project 
Alexandra Buren and Lauren de Silva

## Introduction
An individual decision-maker (agent) is given a set of $X_{1},\dotsc,X_{n}$ objects 
where each object has $m$ possible features $X_{i}=\left\{x_{1,i},\dotsc,x_{m,i}\right\}$.
A utility function ranks the agent's preference for combinations of features for each object $i$:

\begin{equation}
U_{i}(x_{1,i},x_{2,i},\dots,x_{m,i}) = u_{i}
\end{equation}

where $u_{i}$ is a real number called the `utility` for object $i$. Utility has units of `utils`. The utility function $U:X\rightarrow\mathbb{R}$ is unique only up to an order-preserving transformation. Utility functions are `ordinal`, i.e., they rank-order bundles but do not measure differences between bundles.

### Learning objectives
In this example, our goal is to:
* Help students understand how to compute the utility of objects based on their features
* Teach students how to calculate marginal utility and use it to optimize object features
* Introduce the concept of marginal rate of substitution (MRS) and demonstrate how it can be used to understand the trade-offs we make when we are indifferent between features

## Setup

In [25]:
using Pkg
Pkg.add(path="https://github.com/varnerlab/VLDecisionsPackage.jl.git")
Pkg.add("CSV")

    Updating git-repo `https://github.com/varnerlab/VLDecisionsPackage.jl.git`
   Resolving package versions...
  No Changes to `C:\Users\lwdes\CHEME 5760\CHEME5760_finalProject\Project.toml`
  No Changes to `C:\Users\lwdes\CHEME 5760\CHEME5760_finalProject\Manifest.toml`
   Resolving package versions...
    Updating `C:\Users\lwdes\CHEME 5760\CHEME5760_finalProject\Project.toml`
  [336ed68f] + CSV v0.10.11
    Updating `C:\Users\lwdes\CHEME 5760\CHEME5760_finalProject\Manifest.toml`
  [336ed68f] + CSV v0.10.11
  [e2d170a0] + DataValueInterfaces v1.0.0
  [48062228] + FilePathsBase v0.9.21
  [842dd82b] + InlineStrings v1.4.0
  [82899510] + IteratorInterfaceExtensions v1.0.0
  [2dfb63ee] + PooledArrays v1.4.3
  [91c51154] + SentinelArrays v1.4.1
  [3783bdb8] + TableTraits v1.0.1
  [bd369af6] + Tables v1.11.1
  [ea10d353] + WeakRefStrings v1.4.2
  [76eceee3] + WorkerUtilities v1.6.1


In [27]:
include("Include.jl");

  Activating project at `C:\Users\lwdes\CHEME 5760\CHEME5760_finalProject`
    Updating registry at `C:\Users\lwdes\.julia\registries\General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
    Updating git-repo `https://github.com/varnerlab/VLDecisionsPackage.jl.git`
  No Changes to `C:\Users\lwdes\CHEME 5760\CHEME5760_finalProject\Project.toml`
  No Changes to `C:\Users\lwdes\CHEME 5760\CHEME5760_finalProject\Manifest.toml`


In [23]:
# function _load(path::String)::DataFrame
#     return CSV.read(path, DataFrame);
# end

# function _jld2(path::String)::Dict{String,Any}
#     return load(path);
# end

# # short circuit the loading of the data -
# SolarPVSet() = _load(joinpath(_PATH_TO_DATA, "PV vs Solar Heating.csv"));

In [24]:
# SolarPVSet()

LoadError: UndefVarError: `_PATH_TO_DATA` not defined

In [26]:
# # Install the required packages if you haven't already
# # using Pkg
# # Pkg.add("CSV")

# using CSV

# # Replace "your_dataset.csv" with the actual file path
# file_path = "CHEME 5760/CHEME5760_finalProject/data/PV vs Solar Heating.csv"

# # Read the CSV file into a DataFrame
# df = CSV.File(file_path) |> DataFrame

# # Now, 'df' is a DataFrame containing your dataset


LoadError: ArgumentError: "CHEME 5760/CHEME5760_finalProject/data/PV vs Solar Heating.csv" is not a valid file or doesn't exist

In [28]:
function UCD(x)
    
    # load data -
    dataset = SolarPVSet();
    α = dataset[:,:exponent];
    
    number_of_features = length(x);
    solution = 1.0;

    # build the solution -
    for i ∈ 1:number_of_features
        solution *= x[i]^α[i];
    end

    # return - 
    return solution;
end

UCD (generic function with 1 method)

In [29]:
function MUCD(model::VLCobbDouglasUtilityFunction, features::Array{Float64,1})::Array{Float64,1}
    
    # initialize -
    number_of_features = length(features);
    values = zeros(number_of_features);
    α = model.α;
    
    for i ∈ 1:number_of_features
        tmp = prod(features[1:end .!=i].^(α[1:end .!=i]));
        values[i] = tmp*(α[i]*features[i]^(α[i]-1))
    end
    
    # return -
    return values
end

MUCD (generic function with 1 method)

## Data

In [30]:
dataset=SolarPVSet()

LoadError: UndefVarError: `_PATH_TO_DATA` not defined

In [6]:
@assert(sum(dataset[:,:exponent]) == 1.0)

LoadError: UndefVarError: `dataset` not defined

### Build the Cobb-Douglas model
The `Cobb-Douglas` utility function is the product of the $m$ feature variables.  Thus, 
it models situations where we consider features simultaneously. Each feature variable is raised to a non-negative exponent:

\begin{equation}
U(x_{1},\dots,x_{m}) = \prod_{i\in{1\dots{m}}}{x_{i}^{\alpha_{i}}}
\end{equation}

In our realization of the `Cobb-Douglas` utility, the exponents must sum to unity $\sum_{i\in{1\dots{m}}}\alpha_{i} = 1$, $x_{i}\geq{0}$,
and $\alpha_{i}\geq{0}$. For this model, the marginal utility for fearure $i$ is given by:

$$
\bar{U}_{x_{i}} = \left(\alpha_{i}\cdot{x_{i}}^{\alpha_{i}-1}\right)\cdot\left(\prod_{j=1,i}^{m}x_{j}^{\alpha_{j}}\right)
$$

In [7]:
model = build(VLCobbDouglasUtilityFunction, (
        α = dataset[:,:exponent],)
);

LoadError: UndefVarError: `dataset` not defined

In [8]:
utility_tesla = model(dataset[:,:Tesla]);
utility_honda = model(dataset[:,:Honda]);
println("The utility of Tesla = $(utility_tesla) while the utility of the Honda = $(utility_honda)")

LoadError: UndefVarError: `dataset` not defined

### Perception model
Let's suppose we can perceive differences in utility of 1% or greater

In [9]:
Δ = ((max(utility_tesla,utility_honda) - min(utility_tesla,utility_honda))/min(utility_tesla,utility_honda))*100

LoadError: UndefVarError: `utility_tesla` not defined

In [10]:
Ū_tesla = ForwardDiff.gradient(UCD, dataset[:,:Tesla]);
Ū_honda = ForwardDiff.gradient(UCD, dataset[:,:Honda]);

mu_table_df = DataFrame();
for i ∈ eachindex(Ū_tesla)
    
    row_tuple = (
        feature = dataset[i,:feature],
        S_Tesla = dataset[i,:Tesla],
        MU_Tesla = Ū_tesla[i],
        S_Honda = dataset[i,:Honda],
        MU_Honda = Ū_honda[i]
    );

    push!(mu_table_df, row_tuple);
end
mu_table_df

LoadError: UndefVarError: `ForwardDiff` not defined

### Check: How good is the numerical estimate of the marginal utlity?

In [11]:
analytical_MU = MUCD(model,dataset[:,:Tesla]);
numerical_versus_analytical_df = DataFrame();
for i ∈ eachindex(Ū_tesla)
    
    row_tuple = (
        feature = dataset[i,:feature],
        S_Tesla = dataset[i,:Tesla],
        MU_N_Tesla = Ū_tesla[i],
        MU_A_Tesla = analytical_MU[i],
    );

    push!(numerical_versus_analytical_df, row_tuple);
end
numerical_versus_analytical_df

LoadError: UndefVarError: `model` not defined

### Predicting Utility from feature changes
An individual decision-maker (agent) is given a set of $X_{1},\dotsc, X_{n}$ objects 
where each object has $m$ possible features $X_{i}=\left\{x_{1,i} \dotsc ,x_{m,i}\right\}$.
The _local change_ in the utility $U(\dots)$ near a point $x^{\star}\in{X}$ can be computed 
using the _total differential_ of the utility function $U(\dots)$:

\begin{equation}
dU = \sum_{i\in{1\dots{m}}}\bar{U}_{x_{i}}\cdot{dx_{i}}
\end{equation}

where $dU\approx\left(U - U^{\star}\right)$ denotes the local change in utility, 
$\bar{U}_{x_{i}}$ denotes the marginal utility of $x_{i}$ evaluated at the point $x^{\star}\in{X}$, 
and $dx_{i}\approx(x_{i}-x^{\star}_{i})$ is the change in the feature $x_{i}$. 

#### Application
In this example, the `usefulness` feature has the largest marginal utility for Tesla. What would happen to the Model S utility if Tesla were to increase the `usefulness` feature by `+1`? If all else was held constant (`omnibus paribus`), then the change of `dx` in a single feature `i` gives the new utility:

$$
\begin{equation}
U = U^{\star} + \bar{U}_{i}\cdot{dx_{i}}
\end{equation}
$$

In [12]:
dx = zeros(length(Ū_tesla));
dx[4] = 1.0;
U = utility_tesla + sum(Ū_tesla.*dx)

LoadError: UndefVarError: `Ū_tesla` not defined

## Marginal rate of substitution (MRS)
The marginal rate of substitution describes the rate at which a consumer is willing to trade one good (or feature) for another. 
For a utility function $U:X\rightarrow\mathbb{R}$, the marginal rate of substitution between any two goods (or features) $x_{i}$ and $x_{j}$ 
along an \textit{indifference curve} with utility $U^{\star}$, i.e., a curve with utility $U^{\star}$ and $dU = 0$, is given by:

$$
\begin{equation}
\bar{U}_{x_{i}}\,dx_{i} = -\bar{U}_{x_{j}}\,dx_{j}\qquad{\forall{i\neq{j}}}
\end{equation}
$$

or equivalently:

$$
\begin{equation}
\frac{dx_{i}}{dx_{j}} = -\frac{\bar{U}_{x_{j}}}{\bar{U}_{x_{i}}}\qquad{\forall{i\neq{j}}}
\end{equation}
$$

#### What am I willing to give up for a unit of affordability?

In [13]:
dx₂ = 1 ; # change affordability by 1-unit (willing to pay more)
feature_range = [1,3,4,5,6,7]; # we've excluded feature 2
MRS_tradeoff_table = DataFrame();
for i ∈ feature_range
    dx = -(Ū_tesla[2]/Ū_tesla[i])*dx₂
    
    results_row = (
        feature = dataset[i,:feature],
        affordability = dx₂,
        MRS_Tesla = dx
    );
    
    # store -
    push!(MRS_tradeoff_table, results_row);
end

MRS_tradeoff_table

LoadError: UndefVarError: `DataFrame` not defined